In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
import os
from varname import nameof

In [15]:
label_to_playlists = {
    'jazz_playlists'      : 1,
    'classical_playlists' : 2,
    'indie_playlists'     : 3,
    'country_playlists'   : 4,
    'rock_playlists'      : 5,
    'pop_playlists'       : 6,
    'hip_hop_playlists'   : 7,
    'edm_playlists'       : 8
}
label = 1
all_features_df = None
all_labels_df = None
for name, label in label_to_playlists.items():
    playlists_df = None
    path = os.path.join(os.getcwd(), "data", f"{name}_label_{label}.csv")
    dataframe = pd.read_csv(path)
    dataframe = dataframe.drop_duplicates(subset='track_id', keep=False)
    labels = dataframe.pop('label')
    labels = labels.astype('float32')
    features = dataframe.drop(['Unnamed: 0', 'track_id'], axis=1)
    features.astype('float32') 
    if all_features_df is None:
        all_features_df = features
    else:
        all_features_df = pd.concat(
                [all_features_df, features],
                axis=0,
                join="outer",
                ignore_index=False,
                keys=None,
                levels=None,
                names=None,
                verify_integrity=False,
                copy=True,
            )
    if all_labels_df is None:
        all_labels_df = labels
    else:
        all_labels_df = pd.concat(
                [all_labels_df, labels],
                axis=0,
                join="outer",
                ignore_index=False,
                keys=None,
                levels=None,
                names=None,
                verify_integrity=False,
                copy=True,
            )


x_train, x_test, y_train, y_test = train_test_split(all_features_df, all_labels_df, test_size=0.20)



model = keras.models.Sequential([
    keras.layers.Dense(12, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='linear')
])

model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])

history = model.fit(
    x_train,
    y_train,
    steps_per_epoch=20,
    epochs=100,
    validation_steps=5,
    verbose=2)

print(model.evaluate(
    x=x_test,
    y=y_test,
))


Epoch 1/100


C:\Users\fedor\AppData\Roaming\Python\Python39\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


20/20 - 2s - loss: 28.2869 - accuracy: 0.1389 - 2s/epoch - 84ms/step
Epoch 2/100
20/20 - 0s - loss: 28.2869 - accuracy: 0.1389 - 54ms/epoch - 3ms/step
Epoch 3/100
20/20 - 0s - loss: 28.2869 - accuracy: 0.1389 - 54ms/epoch - 3ms/step
Epoch 4/100
20/20 - 0s - loss: 28.2869 - accuracy: 0.1389 - 52ms/epoch - 3ms/step
Epoch 5/100
20/20 - 0s - loss: 28.2869 - accuracy: 0.1389 - 54ms/epoch - 3ms/step
Epoch 6/100
20/20 - 0s - loss: 28.2869 - accuracy: 0.1389 - 50ms/epoch - 2ms/step
Epoch 7/100
20/20 - 0s - loss: 28.2869 - accuracy: 0.1389 - 49ms/epoch - 2ms/step
Epoch 8/100
20/20 - 0s - loss: 28.2869 - accuracy: 0.1389 - 50ms/epoch - 3ms/step
Epoch 9/100
20/20 - 0s - loss: 28.2869 - accuracy: 0.1389 - 56ms/epoch - 3ms/step
Epoch 10/100
20/20 - 0s - loss: 28.2869 - accuracy: 0.1389 - 53ms/epoch - 3ms/step
Epoch 11/100
20/20 - 0s - loss: 28.2869 - accuracy: 0.1389 - 50ms/epoch - 2ms/step
Epoch 12/100
20/20 - 0s - loss: 28.2869 - accuracy: 0.1389 - 52ms/epoch - 3ms/step
Epoch 13/100
20/20 - 0s - 